In [2]:
from logprobs_extraction import *
from utils import *
import pickle

In [3]:
prompts, categories = load_prompts_from_pile()
eval_prompts = prompts[100:300]

In [4]:
samples_df = pickle.load(open('samples_df_final.pkl', 'rb'))
csv_like_ef = pickle.load(open('csv_like_ef.pkl', 'rb'))
csv_like_ef2 = pickle.load(open('csv_like_ef2.pkl', 'rb'))
csv_like_ef3 = pickle.load(open('csv_like_ef3.pkl', 'rb'))

In [6]:
sdfs = []
for i, p in enumerate(eval_prompts):
    df = sample_logprobs(p)
    sdfs.append(df)
    print(p)
samples_df = pd.concat(sdfs)

In [4]:
csv_like_ef = Exfiltrator(EFP1, EFS1, exfiltrate1)
csv_like_ef2 = Exfiltrator(EFP2, EFS2, exfiltrate2)
csv_like_ef3 = Exfiltrator(EFP3, EFS3, exfiltrate3)

In [5]:
for i, p in enumerate(eval_prompts):
    csv_like_ef.get_logprobs(p)
    csv_like_ef2.get_logprobs(p)
    csv_like_ef3.get_logprobs(p)
    print(i)

In [8]:
# import pickle
# pickle.dump(samples_df, open("samples_df_final.pkl", "wb"))
# pickle.dump(csv_like_ef, open("csv_like_ef.pkl", "wb"))
# pickle.dump(csv_like_ef2, open("csv_like_ef2.pkl", "wb"))
# pickle.dump(csv_like_ef3, open("csv_like_ef3.pkl", "wb"))

In [12]:
ef_logprobs1 = csv_like_ef.logprobs.loc[
    csv_like_ef.logprobs['probs'].apply(len) == 5].loc[
    csv_like_ef.logprobs['probs'].apply(lambda x: sum(x.values()) > 0)
]
ef_logprobs2 = csv_like_ef2.logprobs.loc[
    csv_like_ef2.logprobs['probs'].apply(len) == 5].loc[
    csv_like_ef2.logprobs['probs'].apply(lambda x: sum(x.values()) > 0)
]
ef_logprobs3 = csv_like_ef3.logprobs.loc[
    csv_like_ef3.logprobs['probs'].apply(len) == 5].loc[
    csv_like_ef3.logprobs['probs'].apply(lambda x: sum(x.values()) > 0)
]
sampled_logprobs = samples_df.loc[
    samples_df['probs'].apply(len) == 5].loc[
    samples_df['probs'].apply(lambda x: sum(x.values()) > 0)
]
print(len(ef_logprobs1), len(ef_logprobs2), len(ef_logprobs3), len(sampled_logprobs))

190 185 190 182


In [13]:
def make_baseline(probs):
    top_word = sorted(probs, key=lambda x: probs[x])[-1]
    return {
        top_word: 57,
        'xkfdkajdfa': 0,
        'jkajdfkjafkdla': 0,
        'akejrejklra': 0,
        'ajkejfkejfkoea': 0
    }
baseline_logprobs = pd.DataFrame(sampled_logprobs.probs.apply(make_baseline))

In [14]:
agreement_rate_normalized = []
agreement_rate_left = []
agreement_rate_right = []
agreement_rate_lb = []
overlap_rate = []
top1_match_rate = []
top1_present_rate = []

In [15]:
merged = pd.merge(
    sampled_logprobs, ef_logprobs1, left_index=True, right_index=True,
    suffixes=['_true', '_predicted']
)
agreement_rate_normalized.append(merged.apply(
    lambda x: agreement_rate(x['probs_true'], x['probs_predicted'], 'both'), axis=1).values.mean())
agreement_rate_left.append(merged.apply(
    lambda x: agreement_rate(x['probs_true'], x['probs_predicted'], 'left'), axis=1).values.mean())
agreement_rate_right.append(merged.apply(
    lambda x: agreement_rate(x['probs_true'], x['probs_predicted'], 'right'), axis=1).values.mean())
agreement_rate_lb.append(merged.apply(
    lambda x: agreement_rate(x['probs_true'], x['probs_predicted'], None), axis=1).values.mean())
overlap_rate.append(merged.apply(
    lambda x: fraction_overlap(x['probs_true'], x['probs_predicted']), axis=1).values.mean())
top1_match_rate.append(merged.apply(
    lambda x: fraction_top1s_match(x['probs_true'], x['probs_predicted']), axis=1).values.mean())
top1_present_rate.append(merged.apply(
    lambda x: fraction_top1_present(x['probs_true'], x['probs_predicted']), axis=1).values.mean())

In [16]:
merged = pd.merge(
    sampled_logprobs, ef_logprobs2, left_index=True, right_index=True,
    suffixes=['_true', '_predicted']
)
agreement_rate_normalized.append(merged.apply(
    lambda x: agreement_rate(x['probs_true'], x['probs_predicted'], 'both'), axis=1).values.mean())
agreement_rate_left.append(merged.apply(
    lambda x: agreement_rate(x['probs_true'], x['probs_predicted'], 'left'), axis=1).values.mean())
agreement_rate_right.append(merged.apply(
    lambda x: agreement_rate(x['probs_true'], x['probs_predicted'], 'right'), axis=1).values.mean())
agreement_rate_lb.append(merged.apply(
    lambda x: agreement_rate(x['probs_true'], x['probs_predicted'], None), axis=1).values.mean())
overlap_rate.append(merged.apply(
    lambda x: fraction_overlap(x['probs_true'], x['probs_predicted']), axis=1).values.mean())
top1_match_rate.append(merged.apply(
    lambda x: fraction_top1s_match(x['probs_true'], x['probs_predicted']), axis=1).values.mean())
top1_present_rate.append(merged.apply(
    lambda x: fraction_top1_present(x['probs_true'], x['probs_predicted']), axis=1).values.mean())

In [17]:
merged = pd.merge(
    sampled_logprobs, baseline_logprobs, left_index=True, right_index=True,
    suffixes=['_true', '_predicted']
)
agreement_rate_normalized.append(merged.apply(
    lambda x: agreement_rate(x['probs_true'], x['probs_predicted'], 'both'), axis=1).values.mean())
agreement_rate_left.append(merged.apply(
    lambda x: agreement_rate(x['probs_true'], x['probs_predicted'], 'left'), axis=1).values.mean())
agreement_rate_right.append(merged.apply(
    lambda x: agreement_rate(x['probs_true'], x['probs_predicted'], 'right'), axis=1).values.mean())
agreement_rate_lb.append(merged.apply(
    lambda x: agreement_rate(x['probs_true'], x['probs_predicted'], None), axis=1).values.mean())
overlap_rate.append(merged.apply(
    lambda x: fraction_overlap(x['probs_true'], x['probs_predicted']), axis=1).values.mean())
top1_match_rate.append(merged.apply(
    lambda x: fraction_top1s_match(x['probs_true'], x['probs_predicted']), axis=1).values.mean())
top1_present_rate.append(merged.apply(
    lambda x: fraction_top1_present(x['probs_true'], x['probs_predicted']), axis=1).values.mean())

In [19]:
results_df = pd.DataFrame({
    'Top 5 Overlap': overlap_rate,
    'Agreement Rate Lower Bound': agreement_rate_lb,
#     'Agreement Rate Normalized': agreement_rate_left,
    'Top 1 Match Rate': top1_match_rate,
    'Top 1 Presence Rate': top1_present_rate,
})

In [20]:
results_df['Agreement Rate Lower Bound'] = results_df['Agreement Rate Lower Bound'].apply(lambda x: x / 10000)
results_df

,Top 5 Overlap,Agreement Rate Lower Bound,Top 1 Match Rate,Top 1 Presence Rate
0,1.367816,0.262418,0.545977,0.649425
1,1.415205,0.296105,0.561404,0.672515
2,1.000000,0.322990,1.000000,1.000000


In [288]:
print(results_df.to_markdown())

|    |   Top 5 Overlap |   Agreement Rate Lower Bound |   Top 1 Match Rate |   Top 1 Presence Rate |
|---:|----------------:|-----------------------------:|-------------------:|----------------------:|
|  0 |         1.36782 |                     0.262418 |           0.545977 |              0.649425 |
|  1 |         1.4152  |                     0.296105 |           0.561404 |              0.672515 |
|  2 |         1       |                     0.32299  |           1        |              1        |


In [194]:
merged = pd.merge(
    sampled_logprobs, ef_logprobs2, left_index=True, right_index=True,
    suffixes=['_true', '_predicted']
)

In [81]:
print(merged[merged.probs_true.apply(lambda x: list(x.values())[0] < 90)].probs_predicted.apply(
    lambda x: list(x.values())[0]).mean())
print(merged[merged.probs_true.apply(lambda x: list(x.values())[0] > 90)].probs_predicted.apply(
    lambda x: list(x.values())[0]).mean())

64.19242424242422
80.14102564102565
